# Merchant Ranking Algorith
The method of ranking each merchant, will take inspiration from the methods proposed in (https://sapinsider.org/leveraging-analytical-method-for-ranking-suppliers/), in which we rank each key attribute out of 5, then sum them together with weightsget a score for each merchant. 

The Key Attributes for now are: 
- Merchant's BNPL Revenue
- Proportion of 'identified' Fraud transactions
- The customer Base 
- Projected Growth

In [132]:
from pyspark.sql import SparkSession, functions as F
from pyspark.ml import feature as H
# First lets reed the datasets
spark = (
    SparkSession.builder.appName("Data_Explorer")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.driver.memory", "4g")
    .getOrCreate()
)

1. Tansaction_Revenue
For this we will take:
- Total_BNPL_Revenue
- Average_BNPL_Revenue

In [169]:
transactions = spark.read.parquet('../data/processed/transactions')
merchants =  spark.read.parquet('../data/processed/merchants/')
full_dataset = spark.read.parquet('../data/curated/full_dataset/')
final_data_collection = merchants.select('merchant_abn')

In [170]:
final_data_collection = final_data_collection.join(full_dataset.where(F.col('Potential_outlier') == False).groupBy('merchant_abn').agg(F.round(F.sum('BNPL_Revenue'), 2).alias('Total_BNPL_Revenue'), F.round(F.sum('dollar_value'), 2).alias('Total_Dollar_value'), F.count('BNPL_Revenue').alias('Total_Transactions')), on=['merchant_abn'])
# now add last 6 months
final_data_collection = final_data_collection.join(full_dataset.where((F.col('order_datetime') > F.lit('2022-03-01')) & (F.col('Potential_outlier') == False)).groupBy('merchant_abn').agg(F.round(F.sum('BNPL_Revenue'), 2).alias('Total_BNPL_Revenue_6MON'), F.round(F.sum('dollar_value'), 2).alias('Total_Dollar_value_6MON'), F.count('BNPL_Revenue').alias('Total_Transactions_6MON')), on=['merchant_abn'])
# now add fraud attributes
final_data_collection = final_data_collection.join(full_dataset.where(F.col('Potential_outlier') == True).groupBy('merchant_abn').agg(F.round(F.sum('BNPL_Revenue'), 2).alias('Total_BNPL_Revenue_Fraud'), F.count('BNPL_Revenue').alias('Total_Transactions_Fraud'), F.round(F.sum('dollar_value'), 2).alias('Total_Dollar_value_Fraud')), on=['merchant_abn'])
final_data_collection = final_data_collection.join(full_dataset.where((F.col('order_datetime') > F.lit('2022-03-01')) & (F.col('Potential_outlier') == True)).groupBy('merchant_abn').agg(F.round(F.sum('BNPL_Revenue'), 2).alias('Total_BNPL_Revenue_6MON_Fraud'), F.count('BNPL_Revenue').alias('Total_Transactions_6MON_Fraud'), F.round(F.sum('dollar_value'), 2).alias('Total_Dollar_value_6MON_Fraud')), on=['merchant_abn'])

In [171]:
# Get proportion values
final_data_collection = final_data_collection.withColumn('Total_BNPL_Revenue_FP', F.col('Total_BNPL_Revenue_Fraud') / (F.col('Total_BNPL_Revenue') + F.col('Total_BNPL_Revenue_Fraud')))
final_data_collection = final_data_collection.withColumn('Total_Dollar_value_FP', F.col('Total_BNPL_Revenue_Fraud') / (F.col('Total_Dollar_value') + F.col('Total_BNPL_Revenue_Fraud')))
final_data_collection = final_data_collection.withColumn('Total_Transactions_FP', F.col('Total_Transactions_Fraud') / (F.col('Total_Transactions') + F.col('Total_Transactions_Fraud')))
final_data_collection = final_data_collection.withColumn('Total_BNPL_Revenue_6MON_FP', F.col('Total_BNPL_Revenue_6MON_Fraud') / (F.col('Total_BNPL_Revenue_6MON') + F.col('Total_BNPL_Revenue_6MON_Fraud')))
final_data_collection = final_data_collection.withColumn('Total_Dollar_value_6MON_FP', F.col('Total_BNPL_Revenue_6MON_Fraud') / (F.col('Total_Dollar_value_6MON') + F.col('Total_BNPL_Revenue_6MON_Fraud')))
final_data_collection = final_data_collection.withColumn('Total_Transactions_6MON_FP', F.col('Total_Transactions_6MON_Fraud') / (F.col('Total_Transactions_6MON') + F.col('Total_Transactions_6MON_Fraud')))

In [172]:
final_data_collection = final_data_collection.select('merchant_abn', 'Total_BNPL_Revenue', 'Total_Dollar_value', 'Total_Transactions', 'Total_BNPL_Revenue_6MON', 'Total_Dollar_value_6MON', 'Total_Transactions_6MON', 'Total_BNPL_Revenue_FP', 'Total_Dollar_value_FP', 'Total_Transactions_FP', 'Total_BNPL_Revenue_6MON_FP', 'Total_Dollar_value_6MON_FP', 'Total_Transactions_6MON_FP')

In [174]:
for col_name in final_data_collection.columns[1:]:
    values = final_data_collection.select(F.max(col_name).alias('high'), F.min(col_name).alias('low'))
    final_data_collection = final_data_collection.withColumn(col_name, (F.col(col_name) - values.select('low').head()[0]) / (values.select('high').head()[0] - values.select('low').head()[0]))

In [175]:
# Now need to perform Revenue and Risk score
final_data_collection = final_data_collection.withColumn('Revenue_Score', F.col('Total_BNPL_Revenue') + F.col('Total_Dollar_value') + F.col('Total_Transactions') +
                                                  F.col('Total_BNPL_Revenue_6MON') + F.col('Total_Dollar_value_6MON') + F.col('Total_Transactions_6MON'))
# And Risk Revenue score 
final_data_collection = final_data_collection.withColumn('Risk_Revenue_Score', -1 * (F.col('Total_BNPL_Revenue_FP') + F.col('Total_Dollar_value_FP') + F.col('Total_Transactions_FP') +
                                                  F.col('Total_BNPL_Revenue_6MON_FP') + F.col('Total_Dollar_value_6MON_FP') + F.col('Total_Transactions_6MON_FP')))

In [176]:
final_data_collection = final_data_collection.select(['merchant_abn', 'Revenue_Score', 'Risk_Revenue_Score'])

# Merchant Sustainability
Next, we add a rating for a companies growth

In [177]:
merchants_sub = merchants.select(['merchant_abn', 'avg_monthly_inc', 'postcode_entropy'])

In [178]:
for col_name in merchants_sub.columns[1:]:
    values = merchants_sub.select(F.max(col_name).alias('high'), F.min(col_name).alias('low'))
    merchants_sub = merchants_sub.withColumn(col_name, (F.col(col_name) - values.select('low').head()[0]) / (values.select('high').head()[0] - values.select('low').head()[0]))

In [179]:
merchants_sub = merchants_sub.withColumn('Sustainability_score', F.col('avg_monthly_inc') - F.col('postcode_entropy'))

In [181]:
merchants_sub = merchants_sub.select(['merchant_abn', 'Sustainability_score'])
final_data_collection = final_data_collection.join(merchants_sub, on=['merchant_abn'])

# Customer Rankings
In this sections we use the features:
- customer_loyalty_agg
- unique_cust
- Fraud_cust
- Cust_tax

In [198]:
# Create Loyalfy feature
grouped = full_dataset.groupBy("user_id", "merchant_abn")
RPR = grouped.count().withColumnRenamed("count", "RPR")
upSell = RPR.groupBy("user_id").count().withColumnRenamed("count", "upsell")
CLV = grouped.sum("dollar_value").withColumnRenamed("sum(dollar_value)", "CLV")
# Define the window
from pyspark.sql.window import Window
w = Window.partitionBy(["user_id", "merchant_abn"]).orderBy("order_datetime")

retention = full_dataset.withColumn(
    'diff',
    F.datediff(F.col("order_datetime"), F.lag("order_datetime").over(w))
).groupBy("user_id", "merchant_abn").agg(F.avg(F.col("diff")).alias("retention"))
retention.agg({"retention":"max"}).collect()
retention.na.fill(value=365)
loyal = retention.na.fill(value=365).join(RPR, on=["user_id", "merchant_abn"], how="left").join(CLV, on=["user_id", "merchant_abn"], how="left").join(upSell, on=["user_id"], how="left")
loyal = loyal.withColumn("loyal", F.col("RPR") * F.col("CLV") * F.col("upSell") / F.col("retention"))

22/10/06 04:46:51 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/10/06 04:46:51 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/10/06 04:46:52 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/10/06 04:46:52 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/10/06 04:46:52 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


+-------+------------+---------+
|user_id|merchant_abn|retention|
+-------+------------+---------+
|      1| 13118172970|     87.0|
|      1| 16644129035|    365.0|
|      1| 22953464223|    365.0|
+-------+------------+---------+
only showing top 3 rows



In [202]:
from pyspark.sql.functions import col
loyal = loyal.select(*(col(c).cast("float").alias(c) for c in loyal.columns))
loyal = loyal.select('user_id', 'merchant_abn', 'loyal')

In [203]:
# Join to dataset
# full_dataset = full_dataset.join(loyal, on=['user_id', 'merchant_abn'])

In [205]:
customer_attr =  full_dataset.groupBy('merchant_abn').agg(F.countDistinct('user_id').alias('Unique_Customers'), F.round(F.count('user_id') / F.countDistinct('user_id'), 2).alias('Transaction_per_User'), F.round(F.avg('Proportion_Unreg_Merchant_Transactions'),2).alias('Customer_Defects'), F.round(F.avg('Average taxable income or loss'),2).alias('customer_wealth'), F.round(F.avg('loyal'),2).alias('loyalty'))

In [206]:
for col_name in customer_attr.columns[1:]:
    values = customer_attr.select(F.max(col_name).alias('high'), F.min(col_name).alias('low'))
    customer_attr = customer_attr.withColumn(col_name, (F.col(col_name) - values.select('low').head()[0]) / (values.select('high').head()[0] - values.select('low').head()[0]))

22/10/06 05:01:05 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/10/06 05:01:05 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/10/06 05:01:06 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


22/10/06 05:01:41 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/10/06 05:01:41 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/10/06 05:01:42 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


22/10/06 05:03:04 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/10/06 05:03:06 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/10/06 05:03:06 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/10/06 05:03:06 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/10/06 05:03:06 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


22/10/06 05:09:53 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


22/10/06 05:09:57 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


22/10/06 05:10:54 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/10/06 05:10:54 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/10/06 05:10:54 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


22/10/06 05:10:55 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/10/06 05:10:55 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


22/10/06 05:12:13 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/10/06 05:12:13 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/10/06 05:12:13 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/10/06 05:12:13 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


22/10/06 05:12:15 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
